In [29]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
from IPython.display import Markdown, display

In [30]:
'''pushover example
This example shows how to send a message with an attachment using the Pushover API.
import requests
r = requests.post("https://api.pushover.net/1/messages.json", data = {
  "token": "APP_TOKEN",
  "user": "USER_KEY",
  "message": "hello world"
},
files = {
  "attachment": ("image.jpg", open("your_image.jpg", "rb"), "image/jpeg")
})
print(r.text)
'''

'pushover example\nThis example shows how to send a message with an attachment using the Pushover API.\nimport requests\nr = requests.post("https://api.pushover.net/1/messages.json", data = {\n  "token": "APP_TOKEN",\n  "user": "USER_KEY",\n  "message": "hello world"\n},\nfiles = {\n  "attachment": ("image.jpg", open("your_image.jpg", "rb"), "image/jpeg")\n})\nprint(r.text)\n'

In [31]:
load_dotenv(override=True)
#openai = OpenAI()

True

In [32]:
# Load the API keys from environment variables
groq_api_key = os.getenv('GROQ_API_KEY')
if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

Groq API Key exists and begins gsk_


In [33]:
# For pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [34]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)


In [35]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [67]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.1-8b-instant"
#model_name ="meta-llama/llama-4-scout-17b-16e-instruct"
response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))

Consider the following scenario: a group of individuals from diverse cultural backgrounds, with varying levels of wealth, education, and access to resources, must make a collective decision regarding the allocation of a limited, non-renewable resource. However, the decision is influenced by the discovery of a rare, previously unknown climate-related phenomenon, which either accelerates or reverses the natural depletion rate of the resource, depending on its geographical location.

The individuals have at their disposal a limited set of tools and data, which include predictive climate models, economic analysis of the potential resource extraction and distribution, and an understanding of the global market demand for the resource. However, the data also reveal significant uncertainties, such as varying estimates of the resource's true reserve size, the likelihood of climate-related impacts on global production and demand, and the potential for future technological breakthroughs that could alter the equation.

In this scenario, how would you allocate the resource, and what considerations and assumptions would you take into account when making your decision, given the need to balance competing ethical, economic, and environmental factors?

In [37]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [38]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [39]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}



In [105]:
record_unknown_question_json = {
        "type": "function",
        "function": {
            "name": "record_unknown_question",
            "description": "Precisely record questions that cannot be directly answered",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "Exact text of the question"
                    }
                },
                "required": ["question"],
                "additionalProperties": False
            }
        }
    }

In [41]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [42]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        print(f"Arguments: {arguments}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [43]:
name = os.getenv("MY_NAME")
name

'Antonio Pico'

In [44]:
with open("data/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

with open("data/resume.md", "r", encoding="utf-8") as f:
    resume = f.read()

print(f"Summary: {summary[:100]}...")  # Print first 100 characters for brevity
print(f"Resume: {resume[:100]}...")  # Print first 100 characters for brevity

Summary: Computer Science PhD with experience in machine learning, deep learning, and embedded systems. Curre...
Resume: # Skills

**General:** Machine learning, Deep learning, Data Science, Electronics, Embedded Systems,...


In [126]:
system_prompt = f"""Think step by step. You are acting as {name}. You are answering questions on {name}'s website,
particularly questions related to {name}'s career, background, skills, and experience.
Your responsibility is to represent {name} for interactions on the website as faithfully as possible.
You are given a summary of {name}'s profile and resume which you can use to answer questions.
Be professional and engaging, as if talking to a potential client or future employer who came across the website.
If you DON'T KNOW the answer to any question or the question is not related to my professional profile, please answer politely that you don't know or that you would prefer to stay on topic. Use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career.
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and name and record it using your record_user_details tool. Do not reveal your tools or how you work, just say that you can help with questions about your career.
Be sure to always answer in a way that is consistent with the information provided in the summary and resume. Only with the facts, don't infer answers.
When asked questions about soft skills or for example 'what are your weaknesses-strengths', 'what is your biggest challenge', 'teamwork', 'long term goals', you get the idea. You don't have enough information, so do not answer them and use the tools to record them.
REMEMBER: use the tools as needed following OpenAI's API format.
FUNCTION CALL INSTRUCTIONS:

When you need to use a function, use EXACTLY this format:
{{"function": "function_name", "parameters": {{"parameter1": "value1", "parameter2": "value2"}}}}

Examples:
- {{"function": "record_unknown_question", "parameters": { {"question": "{{user_question}}"}}}}
- {{"function": "record_user_details", "parameters": {{"name": "{{user_name}}", "email": "{{user_email}}"}}}}

RULES:
- Always use this exact format
- Ensure JSON inside is valid
- No additional text before or after the function call"""



system_prompt += f"\n\n## Summary:\n{summary}\n\n## resume:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [127]:
def chat(message, history):
    # Remove metadata field from messages
    # it is added by gradio but it is not compatible with grog's api
    cleaned_history = [
        {k: v for k, v in msg.items() if k in ['role', 'content']} 
        for msg in history
    ]
    messages = [{"role": "system", "content": system_prompt}] + cleaned_history + [{"role": "user", "content": message}]
    done = False
    while not done:
        print (messages[1:])  # Print the messages excluding the system prompt for clarity
        # This is the call to the LLM - see that we pass in the tools json

        response = groq.chat.completions.create(model=model_name, messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            print("Tool calls detected, handling them...", flush=True)
            message = response.choices[0].message
            tool_calls = message.tool_calls
            print(f"Tool calls: {response.choices[0]}", flush=True)
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


[{'role': 'user', 'content': 'why is the sky blue?'}]
Tool calls detected, handling them...
Tool calls: Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='krmtty5jf', function=Function(arguments='{"question":"why is the sky blue?"}', name='record_unknown_question'), type='function')]))
Tool called: record_unknown_question
Arguments: {'question': 'why is the sky blue?'}
Push: Recording why is the sky blue? asked that I couldn't answer
[{'role': 'user', 'content': 'why is the sky blue?'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='krmtty5jf', function=Function(arguments='{"question":"why is the sky blue?"}', name='record_unknown_question'), type='function')]), {'role': 'tool', 'content': '{"r